In [1]:
import numpy as np
from lib import ProbabilisticGraph, Node, Edge

# Inference in the Sprinkler model

Remember the Sprinkler model :
<img src="img/sprinkler.png" width="200"/>

Each of the $4$ variables follows a Bernoulli distribution :
- It rained this morning ($rain=1$) with probability $0.2$
- The sprinkler was on ($sprinkler=1$) with probability $0.5$


The following table gives wet grass conditional probability $P(grass=1~|~rain,sprinkler)$:
<table>
    <tr>
        <td></td>
        <td>$rain = 0$</td>
        <td>$rain = 1$</td>
    </tr>
    <tr>
        <td>$sprinkler=0$</td>
        <td>$0.01$</td>
        <td>$0.8$</td>
    </tr>
    <tr>
        <td>$sprinkler=1$</td>
        <td>$0.8$</td>
        <td>$0.95$</td>
    </tr>
</table>

Finally, we also have the following table for the wet dog conditional $P(dog=1~|~grass)$:
<table>
    <tr>
        <td></td>
        <td>$grass = 0$</td>
        <td>$grass = 1$</td>
    </tr>
    <tr>
        <td>$dog=1$</td>
        <td>$0.2$</td>
        <td>$0.7$</td>
    </tr>
</table>


### Question 1 :
- Explain why this model cannot be used as input for a sum-product algorithm.
- How can we fix this ?
- In the following cells, we use `lib.ProbabilisticGraph` to define a tree on which to run the sum-product algorithm. Read and understand the code in the following cells. Complete the TODOs with the correct potentials according to our model.

In [ ]:
p_rain, p_no_rain =   # TODO: complete
p_sprinkled = p_no_sprinkled =   # TODO: complete

In [ ]:
# Create the ProbabilisticGraph object
model = ProbabilisticGraph()

# We add a node to our model for the joint probability of the rain and sprinkler.
rain_sprinkler = model.add_node(
    'rain_sprinkler',  # define a clear name for easier access
    # array of unary potentials for this node
    # TODO: complete the following array
    np.array([
        # P(rain=0, sprinkler=0), P(rain=0, sprinkler=1), P(rain=1, sprinkler=0), P(rain=1, sprinkler=1)
    ])
)

# The grass node
grass = model.add_node('grass', np.array())  # TODO: complete the unary potentials

# The dog node
dog = model.add_node('dog', np.array())  # TODO: complete the unary potentials


In [ ]:
# Now that all the nodes are added, we can add the binary potentials for each edges in the model
model.add_edge(
    # An edge that goes from rain_sprinkler to grass
    rain_sprinkler.id, grass.id,
    # The binary potential A.
    # It is of shape = (|grass|, |rain_sprinkler|)
    # where |X| denotes the number of categorical values of X.
    np.array(),  # TODO: complete
)

# We do the same for grass->dog
model.add_edge(
    grass.id, dog.id,
    np.array()  # TODO: complete
)

### Question 2:
- How can we compute $P(rain, sprinkler | dog=1)$, with the sum product algorithm ?
- Complete the following cells to perform inference, and observe that given an observation of the dog, rain and sprinkler are not independent.
- How many messages are needed to do inference in this graph ? Why ?

In [ ]:
# How can we observe that the dog is wet ?
dog.potentials = np.array([0, 1])  # TODO: complete

# Reset all messages in the graph
model.reset_messages()
# Use sum-product on this graph.
while True:
    # model.send_messages returns the number of messages sent at the current iteration
    n_sent_messages = model.send_messages()
    print(n_sent_messages)
    if n_sent_messages == 0:
        break

In [ ]:
# Node.get_marginal() returns the marginal probabilities for this node, in the order of the unary potentials
marginals, _z = rain_sprinkler.get_marginal()
# marginals = [
#  P(rain=0, sprinkler=0 | dog), P(rain=0, sprinkler=1 | dog),
#  P(rain=1, sprinkler=0 | dog), P(rain=1, sprinkler=1 | dog)  
#]

# From these marginals, compute the marginal probability that it rained, or that the sprinkler was on
proba_rain_on =  # TODO: complete
proba_sprinkler_on =  # TODO: complete

# Observe that given dog was wet, rain and sprinkler are not independent.
print()  # TODO: complete

### Question 3:
How can we see directly on the graph that in general, a distribution that factorizes in this graph does not satisfy conditional independence of $rain$ and $sprinkler$ given $dog$ ?

### Question 4:
Play with the model, and verify a few intuitive properties of the marginal probabilities in the cell below.

# Inference in Phylogenetic Trees

A phylogenetic tree is a tool used in biology to describe relations between species. We will use the following phylogenetic tree :
<img src="img/phylo.png" width="500">

In this picture, nodes correspond to species. Each species can split at a point in time in two sub-species (giving rise to branches in the tree). The length of a branch denotes the expected amount of DNA substitutions (mutation of a single nucleotide) between two species.

We will study the following graphical model on DNA, deduced from this phylogenetic tree :

<img src="img/phylo_pgm.png" width="500">


The random variables in this graphical model are _nucleotides_ (the basic building blocks of DNA). They take categorical values in $\{A, T, G, C\}$. Biologists have defined binary and unary potentials on this graphical model, to model how the DNA might evolve in time.


Your biologist friend recently encountered an <b>unknown</b> species ! Luckily, he was able to sequence it's DNA. As a data-scientist, you proposed to help him update his phylogenetic tree to include this new species.

In the following cell, we load a (synthetic) dataset containing the DNA of 9 species.

In [ ]:
import numpy as np
dataset = np.load('data/dna.npy')
print(dataset.shape)

Each row of this dataset corresponds to the nucleotides observed at a given location in the DNA for different species. More specifically, each row is a vector $X_i \in \{A, T, G, C\}^9$ ($A=0$, $T=1$, $G=2$, $C=3$) with each element of this vector corresponding to the nucleotide at a certain site in the DNA of human, baboon, mouse, rat, cow, pig, cat, dog and the new species, in this order.

In [ ]:
observed = [
    "human", "baboon", "mouse", "rat",
    "cow", "pig", "cat", "dog",
    "observed"
]

In order to update our phylogenetic tree, we need to figure out where a branch needs to be added. We consider that the branch to the new species comes from a new node which is inserted in the middle of some edge of our original model.

<img src="img/phylo_insert_1.png" width="500"/>

For example, in this tree, we added a node <em>new_ancestor</em> between _child2_ and _child5_, leading to _child5_ and our new species _observed_.

However, we don't know where this insertion really happened. There are $14$ trees possible (since there were $14$ edges in the original tree, and the insertion could have happened anywhere).

The following cell generates the corresponding 14 graphical models.

In [ ]:
from lib import gen_candidate_trees

trees = gen_candidate_trees()

# Each key of this dictionary corresponds to the insertion edge
print(trees.keys(), "\n\n")

# Each value of the dictionary is a tuple [ProbabilisticGraph, string]
# We can look at each tree, by printing the string

# notice new_ancestor inserted between child 1 and child 3, with descendant child3 and observed.
print(trees["child1->child3"][1])


To figure out where the insertion happened, we will look at the log-likelihood of our dataset under each of these $14$ models.

For a given probabilistic model $\Psi_m$, the log-likelihood we are interested in is :
$$\mathbb{P}_{\Psi_m}(human=X_{i,0},baboon=X_{i,1},mouse=X_{i,2},rat=X_{i,3},cow=X_{i,4},pig=X_{i,5},cat=X_{i,6},dog=X_{i,7},observed=X_{i,8})$$

However, the sum-product algorithm seen is class was used to compute marginals of a <b>single node</b>. Whereas we are interested here in the marginal-probability of all the leaves of our tree. We will understand in the following questions how this quantity can be found using the sum-product algorithm.

### Question 4 :
Let $H$ be the interior nodes and $L$ the leaves. Assume that we have observed $X_L = x^{obs}_L$ which are <b>fixed</b>.

Consider the _Gibbs_ distribution on $X_H$ (the interior nodes) of the form 
$$q(x_H) = \frac{1}{Z}\prod_{i\in H}\Big[\Psi_{i, \pi_i}(x_i, x_{\pi_i})\Psi_{i}(x_i)\Big]$$
With
$$\Psi_{i, \pi_i}(x_i, x_{\pi_i}) = p(x_i | x_{\pi_i}) \qquad \Psi_{i}(x_i) = \prod_{j\in\text{children}(i) \cap L}\mathbb{P}(X_j=x^{obs}_j|X_i=x_i)$$

- Show that $$q(x_H) = \frac{1}{Z}\Big[\prod_{i\in H}p(x_i|x_{\pi_i})\Big]\Big[\prod_{i\in L}p(X_i=x^{obs}_i|X_{\pi_i}=x_{\pi_i})\Big]$$
- Deduce from this that $$q(x_H) = \frac{1}{Z}\mathbb{P}(X_H=x_H, X_L=x_L^{obs})$$
- Deduce from this that $\mathbb{P}(X_L=x^{obs}_L) = Z$
- How can you use Sum-Product to compute $\mathbb{P}(X_L=x^{obs}_L)$ ?

In [ ]:
avg_log_likelihood = []
edges = []
for tree, (proba_graph, _string) in trees.items():
    log_likelihoods = []
    for row_id, dna_site in enumerate(dataset):
        # For each of the 9 species, update the potentials to observe the nucleotides
        for node_name, nucleotide in zip(observed, dna_site):
            # Get the Node object from the probabilistic graph
            node = proba_graph.get_node(node_name)
            # Set its potentials to 0 everywhere except for the observed nucleotide
            node.potentials *= 0
            node.potentials[int(nucleotide)] = 1
        #Reset the messages
        proba_graph.reset_messages()
        
        # Run inference in this probabilistic graph
        while proba_graph.send_messages():
            pass
        # P(X_0, ..., X_9) = Z
        # we can compute this normalizing factor anywhere in the hidden nodes.
        _, Z_dna_site = proba_graph.get_node("root").get_marginal()

        log_likelihoods.append(np.log(Z_dna_site))
        
        # Reset the potentials for this graph
        for node_name in observed:
            node = proba_graph.get_node(node_name)
            node.potentials.fill(1)
        
    # Get average log-likelihood
    avg_log_likelihood.append(np.mean(log_likelihoods))
    edges.append(tree)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(dpi=150)
plt.xticks(rotation='vertical')
plt.bar(edges, avg_log_likelihood)
plt.show()

### Question 4:
- According to this figure, what is the most likely edge where the insertion happened.

### Question 5:
- Describe how you would obtain the most likely DNA for the ancestor of the species you just discovered ? In particular, which quantities do you need to compute ? How can they be computed ?
- Implement this method in the next cell. The corresponding node in the probabilistic graph is named `new_ancestor`. Include its first 10 nucleotides (in numerical form) in your report.

### Note :
- Feel free to look at lib.py to see how I implemented the sum-product algorithm. The relevant classes are `Edge`, `Node` and `ProbabilisticGraph`. In `ProbabilisticGraph`, you can focus on `send_messages`. In `Node` all the methods are related to the SumProduct Algorithm